
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 1 - Search Algorithms <br>
<font color=696880 size=4>
Amir Mohammad Fakhimi


____

# Personal Data

In [ ]:
# Set your student number and name
student_number = '401106096'
Name = 'Radin'
Last_Name = 'Shahdaei'

# Rules

<font color=red>
Please run all the cells.
</font>

# Libraries

In [ ]:
# import libraries here

from abc import abstractmethod
import numpy as np
from numpy import random
import pandas as pd
from collections import deque

# Q1: Uninformed Search (DFS, BFS, IDS, A*) (100 Points)

In this jupyter notebook, we aim to implement four uninformed search algorithms: DFS, BFS, IDS, and A*. We also implement a maze class to test the algorithms.

## Tile
Each tile is one maze's square. Each part has a (x, y) coordinate. Also it has g (the cost until now), h (heuristic) and f (sum of g and h).

In [ ]:
class Tile:
    def __init__(self, x, y):
        self.x = x
        self.y = y

        self.g = 0
        self.h = 0
        self.f = 0

        self.parent = None

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __repr__(self):
        return f'({self.x}, {self.y})'

## Maze
It has a map attribute. It's a 2d array of a string type. Each index is one of the following strings:
- '0': A free index which the agent can move to.
- '1': A wall which the agent can't move to.
- 'S': The start point of the agent.
- 'E': The end point of the agent.
- 'V': The tiles that the agent has visited. It is used in the [visualizer website](https://project.amfakhimi.com/maze-visualizer), not in this jupyter notebook.
- 'P': The path that the agent has moved through. It appears after the agent has found the path in the `print_path` function of the `Algorithm` class.

It has three types of initialization:
- a tuple (a, b): a is the height of the map, and b is the width of the map. It generates a random map with 80% of '0' and 20% of '1' and sets the start and end points randomly. It is suggested to use it after setting the `random.seed` to a constant number.
- a string: the path of a csv file which contains the map. You can export your customized map from [here](https://project.amfakhimi.com/maze-visualizer).
- a list or a numpy array: the map itself. It should have 'S' and 'E' in it.

The most important methods are:
- `get_tile`: It returns the value of the tile in the given (x, y) coordinate. Pay attention that the first index is y and the second one is x.
- `export_map`: It exports the map to a csv file. You don't need to use it in this assignment.
- `print_map`: It prints the map in the console to visualize it.

You can create your own map [here](https://project.amfakhimi.com/maze-visualizer).  Before exporting the map, clear the map, first.

In [ ]:
class Maze:
    def __init__(self, map, name):
        self.name = name

        if type(map) == tuple:
            self.height = map[0]
            self.width = map[1]
            self.map = np.zeros((self.height, self.width), dtype=str)
            self.start = Tile(0, 0)
            self.end = Tile(self.width - 1, self.height - 1)

            for y in range(self.height):
                for x in range(self.width):
                    if x == self.start.x and y == self.start.y:
                        self.map[y][x] = 'S'
                    elif x == self.end.x and y == self.end.y:
                        self.map[y][x] = 'E'
                    else:
                        self.map[y][x] = random.choice(['0', '1'], p=[0.8, 0.2])

        elif type(map) == str:
            map_csv = pd.read_csv(map, header=None, dtype=str)
            self.map = map_csv.values
            for y, row in enumerate(self.map):
                for x, char in enumerate(row):
                    if char == 'V' or char == 'P':
                        self.map[y][x] = '0'

        elif type(map) == list or type(map) == np.ndarray:
            self.map = np.array(map)
        else:
            raise ValueError('Invalid map type')

        self.width = len(self.map[0])
        self.height = len(self.map)
        self.initial_map = map

        self.start = None
        self.end = None
        for y, row in enumerate(self.map):
            for x, char in enumerate(row):
                if char == 'S':
                    self.start = Tile(x, y)
                elif char == 'E':
                    self.end = Tile(x, y)

        if self.start is None or self.end is None:
            raise ValueError('Map must contain a start and end point')

    def set_tile(self, x, y, value):
        if value != 'S' and value != 'E' and value != 'P' and value != '0' and value != '1':
            raise ValueError('Invalid tile value')

        self.map[y][x] = value

    def get_tile(self, x, y):
        return self.map[y][x]

    def export_map(self, file_path):
        csv = pd.DataFrame(self.map)
        if file_path is not None:
            csv.to_csv(file_path, index=False, header=False)

        return csv

    def print_map(self):
        for row in self.map:
            print(''.join([str(x) for x in row]))

## Algorithms (DFS, BFS, IDS, A*) (80 Points)
It has the following attributes:
- `maze`: The maze object.
- `open_list`: A list of tiles that are not visited yet and we get the tile with lowest f by iterating through it.
- `closed_list`: A list of tiles that are visited.
- `path`: The path that the agent has found.

The most important methods are:
- `get_neighbors` (You should implement it -> 10 points): It returns the neighbors of the given tile.
- `solve` (You should implement it -> 70 points): It solves the maze. You should use and fill the `open_list` and `closed_list` and `path` attributes. You should also determine if there's no path to the end point. It is an abstract method, and you should implement it in the subclasses. DFS has 10 points, BFS has 10 points, IDS has 20 points, and A* has 30 points. It has 70 points in total.
- `print_path`: It prints the map with the path that the agent has found.
- `export_closed_list`: It exports the closed list to a csv file. You don't need to use it in this assignment directly.
- `export_path`: It exports the path to a csv file. You don't need to use it in this assignment directly.

You should implement the `get_neighbors` in `Algorithm` class and `solve` in the subclasses (DFS, BFS, IDS, A*). You are free to add any other methods or attributes to the classes. (10 + 70 = 80 points)

In [ ]:
class Algorithm:
    def __init__(self, maze):
        self.maze = maze
        self.open_list = []
        self.closed_list = []
        self.path = []

    def get_neighbors(self, tile):
        neighbors = []
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for dx, dy in directions:
            neighbor_x = tile.x + dx
            neighbor_y = tile.y + dy

            if 0 <= neighbor_x < self.maze.width and 0 <= neighbor_y < self.maze.height:
                neighbor_tile = Tile(neighbor_x, neighbor_y)
                neighbors.append(neighbor_tile)

        return neighbors

    def get_tile_from_list(self, tile, tile_list):
        for t in tile_list:
            if t == tile:
                return t

        return None

    def get_tile_from_open_list(self, tile):
        return self.get_tile_from_list(tile, self.open_list)

    def get_tile_from_closed_list(self, tile):
        return self.get_tile_from_list(tile, self.closed_list)

    def is_tile_in_list(self, tile, tile_list):
        for t in tile_list:
            if t == tile:
                return True
        return False

    def is_tile_in_open_list(self, tile):
        return self.is_tile_in_list(tile, self.open_list)

    def is_tile_in_closed_list(self, tile):
        return self.is_tile_in_list(tile, self.closed_list)

    def is_tile_walkable(self, tile):
        return self.maze.get_tile(tile.x, tile.y) != '1'

    @abstractmethod
    def solve(self):
        pass

    def print_path(self, path):
        if path is None:
            print('No path found')
            return

        for y, row in enumerate(self.maze.map):
            for x, char in enumerate(row):
                if Tile(x, y) in path and char != 'S' and char != 'E':
                    print('P', end='')
                else:
                    print(char, end='')
            print()

    def export_closed_list(self, file_path):
        closed_list = [[]]
        for tile in self.closed_list:
            closed_list[0].append(tile.x)
            closed_list[0].append(tile.y)

        csv = pd.DataFrame(closed_list)
        if file_path is not None:
            csv.to_csv(file_path, index=False, header=False)

        return csv

    def export_path(self, file_path):
        path = [[]]
        for tile in self.path:
            path[0].append(tile.x)
            path[0].append(tile.y)

        csv = pd.DataFrame(path)
        if file_path is not None:
            csv.to_csv(file_path, index=False, header=False)
        return csv

    def construct_path(self, end_tile):
        self.path = []
        current_tile = end_tile
        while current_tile:
            self.path.insert(0, current_tile)
            current_tile = current_tile.parent

    def accept_neighbor(self, neighbor):
      condition1 = not self.is_tile_in_closed_list(neighbor)
      condition2 = self.is_tile_walkable(neighbor)
      condition3 = neighbor not in self.open_list
      return condition1 and condition2 and condition3

In [ ]:
class DFS(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)

    def solve(self):
        start_tile = self.maze.start
        end_tile = self.maze.end

        self.open_list.append(start_tile)

        while self.open_list:
            current_tile = self.open_list.pop()

            if current_tile == end_tile:
                self.construct_path(current_tile)
                return True

            self.closed_list.append(current_tile)
            neighbors = self.get_neighbors(current_tile)

            for neighbor in neighbors:
                if self.accept_neighbor(neighbor):
                    neighbor.parent = current_tile
                    self.open_list.append(neighbor)

        return False

In [ ]:
class BFS(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)

    def solve(self):
        start_tile = self.maze.start
        end_tile = self.maze.end

        self.open_list = deque([start_tile])

        while self.open_list:
            current_tile = self.open_list.popleft()

            if current_tile == end_tile:
                self.construct_path(current_tile)
                return True

            self.closed_list.append(current_tile)
            neighbors = self.get_neighbors(current_tile)

            for neighbor in neighbors:
                if self.accept_neighbor(neighbor):
                    neighbor.parent = current_tile
                    self.open_list.append(neighbor)

        return False


In [ ]:
class IDS(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)

    def solve(self):
        start_tile = self.maze.start
        end_tile = self.maze.end

        max_depth = self.maze.height * self.maze.width

        for depth_limit in range(1, max_depth + 1):
            start_tile.g = 0

            self.open_list = [start_tile]
            self.closed_list = []

            while self.open_list:
                current_tile = self.open_list.pop()

                if current_tile == end_tile:
                    self.construct_path(current_tile)
                    return True

                self.closed_list.append(current_tile)

                if current_tile.g >= depth_limit:
                    continue

                neighbors = self.get_neighbors(current_tile)

                for neighbor in neighbors:
                    if self.accept_neighbor(neighbor):
                        neighbor.g = current_tile.g + 1
                        neighbor.parent = current_tile
                        self.open_list.append(neighbor)


        return False

In [ ]:
class AStar(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)

    def get_tile_with_lowest_f(self):
        lowest_f = self.open_list[0]

        for tile in self.open_list:
            if tile.f < lowest_f.f:
                lowest_f = tile

        return lowest_f

    def get_h(self, tile):
        manhattan_distance = abs(tile.x - self.maze.end.x) + abs(tile.y - self.maze.end.y)
        return manhattan_distance

    def get_f(self, tile):
        return tile.g + tile.h

    def solve(self):
        start_tile = self.maze.start
        end_tile = self.maze.end

        self.open_list.append(start_tile)

        while self.open_list:
            current_tile = self.get_tile_with_lowest_f()

            if current_tile == end_tile:
                self.construct_path(current_tile)
                return True

            self.open_list.remove(current_tile)
            self.closed_list.append(current_tile)

            neighbors = self.get_neighbors(current_tile)

            for neighbor in neighbors:
                if not self.accept_neighbor(neighbor):
                    continue

                tentative_g = current_tile.g + 1

                neighbor.parent = current_tile

                neighbor.g = tentative_g
                neighbor.h = self.get_h(neighbor)
                neighbor.f = self.get_f(neighbor)

                self.open_list.append(neighbor)

        return False

The following cell exports all needed data. You can simply run it and upload its output in the [visualizer site](https://project.amfakhimi.com/maze-visualizer).

In [ ]:
def export_all(maze, solver, merge_file_path):
    map_csv = maze.export_map(None)
    closed_list_csv = solver.export_closed_list(None)
    path_csv = solver.export_path(None)

    merge = pd.concat([closed_list_csv, path_csv, map_csv], axis=0)
    merge.to_csv(merge_file_path, index=False, header=False)

## Test (20 points)
if you want to visualize the map, you can use `export_all` function and upload the output in the [visualizer site](https://project.amfakhimi.com/maze-visualizer).

<br>
<font size=5>
After running below tests with seed 0 and (20, 20) as the maze's size, attach your outputs (4 outputs) to the final answer in Quera.
<br>
Your answers based on your implementations could be different from the expected answers, below. So, don't worry about the correctness of your answer. Just make sure that your code works correctly and you have attached the outputs.

In [ ]:
random.seed(0)
maze = Maze((20, 20), 'map')
# maze = Maze('./Map.csv', 'map')

maze.print_map()

S0000000110000100010
11000000100000000001
00000000000001000000
00000001010101000000
00000000001000000001
00001000001010010101
00010000000000000000
01001100101010001100
00001110000000000010
00000100000000100000
00000000001100000000
10100010010000000000
01000000000010010001
00001000000110101100
10100001000001000011
11000000000000000001
00000000000000000000
00000000001000000000
10100100010000000010
0101011100000010100E


In [ ]:
solver = DFS(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'dfs.csv')

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (7, 1), (7, 2), (6, 2), (5, 2), (4, 2), (3, 2), (2, 2), (1, 2), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (9, 5), (9, 6), (8, 6), (7, 6), (7, 7), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (17, 9), (18, 9), (19, 9), (19, 10), (19, 11), (18, 11), (17, 11), (16, 11), (15, 11), (14, 11), (13, 11), (12, 11), (11, 11), (10, 11), (10, 12), (9, 12), (8, 12), (7, 12), (6, 12), (5, 12), (4, 12), (3, 12), (3, 13), (3, 14), (4, 14), (5, 14), (6, 14), (6, 15), (7, 15), (8, 15), (9, 15), (10, 15), (11, 15), (12, 15), (13, 15), (14, 15), (15, 15), (16, 15), (17, 15), (18, 15), (18, 16), (19, 16), (19, 17), (19, 18), (19, 19)]
SPPPPPPP110000100010
1100000P100000000001
PPPPPPPP000001000000
P0000001010101000000
PPPPPPPPPP1000000001
000010000P1010010101
0001000PPP0000000000
0100110P101010001100
0000111PPPPPPPPPPP10
00000100000000100PP

In [ ]:
solver = BFS(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'bfs.csv')

[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (4, 16), (5, 16), (6, 16), (7, 16), (8, 16), (9, 16), (10, 16), (11, 16), (11, 17), (11, 18), (12, 18), (13, 18), (14, 18), (15, 18), (16, 18), (17, 18), (17, 19), (18, 19), (19, 19)]
SPP00000110000100010
11P00000100000000001
00P00000000001000000
00P00001010101000000
00P00000001000000001
00P01000001010010101
00P10000000000000000
01P01100101010001100
00P01110000000000010
00P00100000000100000
00PP0000001100000000
101P0010010000000000
010P0000000010010001
000P1000000110101100
101P0001000001000011
110P0000000000000001
000PPPPPPPPP00000000
00000000001P00000000
10100100010PPPPPPP10
01010111000000101PPE


In [ ]:
solver = IDS(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'ids.csv')

In [ ]:
solver = AStar(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'a_star.csv')

[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (4, 16), (5, 16), (6, 16), (7, 16), (8, 16), (9, 16), (10, 16), (11, 16), (11, 17), (11, 18), (12, 18), (13, 18), (14, 18), (15, 18), (16, 18), (17, 18), (17, 19), (18, 19), (19, 19)]
SPP00000110000100010
11P00000100000000001
00P00000000001000000
00P00001010101000000
00P00000001000000001
00P01000001010010101
00P10000000000000000
01P01100101010001100
00P01110000000000010
00P00100000000100000
00PP0000001100000000
101P0010010000000000
010P0000000010010001
000P1000000110101100
101P0001000001000011
110P0000000000000001
000PPPPPPPPP00000000
00000000001P00000000
10100100010PPPPPPP10
01010111000000101PPE
